In [50]:
import pandas as pd
import numpy as np
import numpy.random as npr

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [60]:
integer_list=[]
for m in range(25):
    integer_list.append(random.randint(1,10))
#print(integer_list)
int_file="data/data_int.txt"
with open(int_file,'w') as file:
    for i in range(0,len(integer_list)):
        file.write(str(integer_list[i])+" ")       
print("\nThe integer list is:")
!cat data/data_int.txt

float_matrix=npr.random((5,5))
float_file="data/data_float.txt"
with open(float_file,'w') as f_file:
    for k in range(0,len(float_matrix)):
        f_file.write(str(float_matrix[k])+"\n")       
print("\nThe float matrix is:")
!cat data/data_float.txt


The integer list is:
9 1 1 1 3 9 9 7 9 9 1 8 7 6 6 8 10 7 1 5 2 4 2 8 6 
The float matrix is:
[0.39893425 0.92533093 0.88076016 0.83967869 0.31084581]
[0.98077106 0.10139062 0.83561983 0.18533866 0.24400738]
[0.24723728 0.91366445 0.75158432 0.24035598 0.20622766]
[0.99789614 0.75253871 0.89918714 0.56380526 0.59112746]
[0.74038273 0.32992036 0.37209459 0.18813847 0.62035444]


2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?